# A gradio webapp with Azure ChatGPT

<img src="bot.png">

In [ ]:
import openai
import os
import gradio as gr
import openai
import random
import time
import sys

# from dotenv import load_dotenv

In [ ]:
sys.version

Loading configuration from GH Secrets

In [ ]:
#load_dotenv("azure.env")

openai.api_type: str = "azure"
# openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")
# openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
# openai.api_version = os.getenv("OPENAI_API_VERSION")
openai.api_version = os.getenv("AZURE_OPENAI_MODEL_CHAT_VERSION")

In [ ]:
# deployment_id = "gpt-35-turbo-16k"
deployment_id = os.getenv("AZURE_OPENAI_MODEL_CHAT") # the GH secret contains you gpt deployment name

In [ ]:
system_message = {"role": "system", "content": "You are an AI assistant."}

# https://huggingface.co/spaces/gradio/theme-gallery

with gr.Blocks() as azure_openai_bot:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    state = gr.State([])

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history, messages_history):
        user_message = history[-1][0]
        bot_message, messages_history = ask_gpt(user_message, messages_history)
        messages_history += [{"role": "assistant", "content": bot_message}]
        history[-1][1] = bot_message
        time.sleep(1)
        return history, messages_history

    # the definition that makes the call to the OpenAI ChatCompletion API sending the user message + history
    def ask_gpt(message, messages_history):
        messages_history += [{"role": "user", "content": message}]
        response = openai.ChatCompletion.create(
            engine=deployment_id,
            messages=messages_history
        )
        print(response["choices"][0]["message"]["content"], messages_history)
        return response["choices"][0]["message"]["content"], messages_history

    def init_history(messages_history):
        messages_history = []
        messages_history += [system_message]
        return messages_history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, [chatbot, state], [chatbot, state]
    )

    clear.click(lambda: None, None, chatbot, queue=False).success(
        init_history, [state], [state]
    )

If execute in GitHub Codespace, a pop-up window will propose to open the website. You can also open it clicking in **Ports** next to Terminal tab in the codespace, look for port 7860 and **Open in Browser**.

In [ ]:
azure_openai_bot.launch(share=True)

In [ ]:
azure_openai_bot.close()